In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  

%matplotlib inline

from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

data_dir = Path('/home/hadim/.data/Neural_Network/captchanet')

dataset_dir = data_dir / 'dataset_v6'
train_data_dir = dataset_dir / 'training'
val_data_dir = dataset_dir / 'validation'

model_dir = data_dir / 'model'
model_name = 'v8'
model_path = model_dir / model_name

tokenizer_path = dataset_dir / "tokenizer.json"

# Get tokenizer
with open(tokenizer_path) as f:
  #tokenizer = tf.keras.preprocessing.text.tokenizer_from_json(f.read())
  from keras_preprocessing import text
  tokenizer = text.tokenizer_from_json(f.read())

In [ ]:
# Load model from checkpoints
log_name = "2019.07.25-10.39.55"
checkpoint_name = 'weights.hdf5'
log_path = data_dir / 'log' / log_name / 'checkpoints' / checkpoint_name

model = tf.keras.models.load_model(str(log_path))

In [ ]:
# Save model
model_path.mkdir(exist_ok=True)
model.save(str(model_path / 'model'))

# Save tokenizer
with open(model_path / tokenizer_path.name, 'w') as f:
  f.write(tokenizer.to_json())
  
# Save history
history = pd.read_csv(log_path.parents[1] / 'log.csv')
history_path = model_path / 'history.csv'
history.to_csv(str(history_path), index=False)

In [ ]:
# Check history

history_path = model_path / 'history.csv'
history = pd.read_csv(history_path)

n = 3
size = 3.5
ncols = 3
w_h_scale = 2
figsize = (ncols * size * w_h_scale, size)
fig, axs = plt.subplots(nrows=n//ncols, ncols=ncols, figsize=figsize)
axs = axs.flatten()

epoch_start = 0
axs[0].plot(history['val_accuracy'][epoch_start:], label='val_accuracy')
axs[0].plot(history['accuracy'][epoch_start:], label='accuracy')
axs[0].set_xlabel('epoch')
axs[0].legend()

axs[1].plot(history['val_loss'][epoch_start:], label='val_loss')
axs[1].plot(history['loss'][epoch_start:], label='loss')
axs[1].set_xlabel('epoch')
axs[1].legend()

axs[2].plot(history['lr'][epoch_start:])
axs[2].set_xlabel('epoch')

fig.savefig(model_path / 'history.png')